# DenseNet-169 for Multi-class Classification of CXR Images

### Objectives

The aim of this notebook is to train variations of the DenseNet arhictecture on the Chest X-ray (Pneumonia) data set sourced from Kaggle. The images are composed of CXR scans representing normal lungs, bacterial pneumonia, and viral pneumonia.

A variable classification head will be built on top of the chosen network, in which varying hyperparameters will be configured and evaluated. The experiment tracking will be conducted with the *Weights and Biases* platform. The optimal model will be selected based on a trade-off evaluation between **MCC score** and **validation loss**. 

### Machine Configurations

`GPU` : NVIDIA GeForce RTX 4090;
`CPU` : AMD Ryzen 7 3700X 8-Core Processor;
`VRAM` : 24.0 GB

### Load all required libraries:

In [1]:
import os
import matplotlib.pyplot as plt

import numpy as np
from sklearn.metrics import f1_score, matthews_corrcoef

import tensorflow as tf

# Suppress TensorFlow messages:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print("TensorFlow version : ", tf.__version__)  # check TensorFlow version
gpu = tf.config.list_physical_devices("GPU")    # check if TensorFlow is using the GPU
print(gpu)

# Enable memory growth for GPU:
try:
   tf.config.experimental.set_memory_growth(gpu[0], True)
   print("Memory growth set.")
except:
    print("GPU runtime already initialised.")

TensorFlow version :  2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memory growth set.


### Configure experiment:

In [2]:
# Configure notebook name for WandB:
%env "WANDB_NOTEBOOK_NAME" "PCXR-evaluating-pretrained-cnn-model"
import wandb
from wandb.keras import WandbMetricsLogger
wandb.login()

env: "WANDB_NOTEBOOK_NAME"="PCXR-evaluating-pretrained-cnn-model"


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marcus-lim. Use `wandb login --relogin` to force relogin


True

In [3]:
# Choose method for hyperparameter selection:
sweep_config = {
    "method": "bayes"
    }

In [4]:
# Establish objective to optimise:
metric = {
    "name": "val_loss",
    "goal": "minimize"   
    }

sweep_config["metric"] = metric

In [5]:
# Choose hyperparameters to sweep:
param_dict = {
    "batch_size": {
        "values": [16, 32, 64]
        },
    "optimiser": {
        "values": ["adam", "sgd"]
        },
    "fc_layer_size": {
        "values": [32, 64, 128, 256]
        },
    "dropout_1": {
        "values": [0.2, 0.6, 0.8]
        },
    "dropout_2": {
        "values": [0.2, 0.6, 0.8]
        },
    "dropout_3": {
        "values": [0.2, 0.6, 0.8]
        },
    }

sweep_config["parameters"] = param_dict

In [6]:
# Instantiate constant value for epoch:
param_dict.update({
    "epochs": {
        "value": 10}
    })

In [7]:
# Use uniform distribution for learning rate:
param_dict.update({
    "learning_rate": {
        "distribution": "uniform",
        "min": 0.0001,
        "max": 0.005
      }
    })

In [8]:
import pprint

pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'values': [16, 32, 64]},
                'dropout_1': {'values': [0.2, 0.6, 0.8]},
                'dropout_2': {'values': [0.2, 0.6, 0.8]},
                'dropout_3': {'values': [0.2, 0.6, 0.8]},
                'epochs': {'value': 10},
                'fc_layer_size': {'values': [32, 64, 128, 256]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.005,
                                  'min': 0.0001},
                'optimiser': {'values': ['adam', 'sgd']}}}


### Load data:

In [119]:
def build_datasets(train_dir="..\\artifacts\\train",
                   test_dir="..\\artifacts\\test",
                   class_names=["normal", "bacteria", "virus"],
                   batch_size=32,
                   image_size=(300, 300),
                   val_split=0.2):
    
    train_ds, valid_ds = tf.keras.utils.image_dataset_from_directory(
        directory=train_dir,
        label_mode="int",
        class_names=class_names,
        batch_size=batch_size,
        image_size=image_size,
        shuffle=True,
        seed=42,
        validation_split=val_split,
        subset="both")

    test_ds = tf.keras.utils.image_dataset_from_directory(
        directory=test_dir,
        label_mode="int",
        class_names=class_names,
        batch_size=batch_size,
        image_size=image_size,
        shuffle=True,
        seed=42,
        subset=None)
    
    return (train_ds, valid_ds, test_ds)

### Construct network:

In [120]:
def build_network(fc_layer_size=128, dropout_1=0.2, dropout_2=0.2, dropout_3=0.2):

    # Configure image shape for model input shape
    IMG_SHAPE = (300, 300, 3)

    # Instantiate base model
    base_model = tf.keras.applications.densenet.DenseNet169(
        input_shape=IMG_SHAPE,
        include_top=False,
        weights="imagenet")

    # Freeze convolutional base
    base_model.trainable = False

    inputs = tf.keras.Input(shape=IMG_SHAPE)

    # Add an augmentation layer for additional training data
    augment_layer = tf.keras.Sequential([
        tf.keras.layers.RandomContrast(factor=0.2, seed=42),
        tf.keras.layers.RandomBrightness(factor=0.2, seed=42),
        tf.keras.layers.RandomRotation(factor=0.01, seed=42)
    ])

    # Establish base model architecture:
    x = augment_layer(inputs)
    x = tf.keras.applications.densenet.preprocess_input(x)
    x = base_model(x, training=False)    # Prevent training of BN layers

    # Add custom classification head:
    x = tf.keras.layers.MaxPooling2D(input_shape=(1664,))(x)
    x = tf.keras.layers.Dropout(dropout_1)(x)  # Prevent overfitting
    x = tf.keras.layers.Dense(fc_layer_size, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_2)(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(dropout_3)(x)

    outputs = tf.keras.layers.Dense(3, activation="softmax")(x)
    return tf.keras.Model(inputs, outputs)

### Define optimiser:

In [121]:
def build_optimiser(learning_rate=0.0001, optimiser="adam"):
    if optimiser.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=learning_rate)
    if optimiser.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=learning_rate)

### Create callback to log custom metrics:

In [122]:
# Compute MCC, F1, and AUC scores and log in WandB:
class CustomLogCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, x_val, y_val):
        super().__init__()
        self.model = model
        self.x_val = x_val
        self.y_val = y_val

        # Instantiate standalone metrics:
        self._mcc = tf.keras.metrics.Mean(name="mcc")
        self._f1score = tf.keras.metrics.Mean(name="f1_score")
        self._aucroc = tf.keras.metrics.AUC(name="auc_roc")

        self.epoch = 0

    def on_epoch_end(self, epoch, logs=None):
        self.epoch += 1

        self._mcc.reset_state()
        self._f1score.reset_state()
        self._aucroc.reset_state()

        print("Generating predictions and computing metrics for Epoch {} ".format(self.epoch))
        predictions = self.model.predict(self.x_val)

        f1score = f1_score(self.y_val, np.argmax(predictions, axis=-1),
                           average=None)
        mcc = matthews_corrcoef(self.y_val, np.argmax(predictions, axis=-1))

        self._mcc.update_state(mcc)
        self._f1score.update_state(f1score)
        self._aucroc.update_state(self.y_val, np.argmax(predictions, axis=-1))

        print("training loss : {} , training acc : {} , mcc score : {}".format(
            logs["loss"], logs["accuracy"], self._mcc.result().numpy()
        ))
        print("aucroc score  : {} , f1 score     : {} ".format(
            self._aucroc.result().numpy(), self._f1score.result().numpy()
        ))

        # Log metrics to WandB:
        wandb.log({"mcc": self._mcc.result().numpy(),
                   "fmeasure": self._f1score.result().numpy(),
                   "auc_roc": self._aucroc.result().numpy()})

### Create training function:

In [131]:
# Build data sets outside of train function:
train_ds, valid_ds, _ = build_datasets(batch_size=None)

# Convert validation data to arrays for custom metric computation:
x_val = np.reshape(np.concatenate([x for x, _ in valid_ds], axis=0), (936, 300, 300, 3))
y_val = np.concatenate(np.reshape([y for _, y in valid_ds], (1, 936)), axis=0)

def train(model, batch_size=32, epochs=10, learning_rate=0.0001, optimiser_name="adam"):

    tf.keras.backend.clear_session()
    train_ds, valid_ds, _ = build_datasets(batch_size=batch_size)
    optimiser = build_optimiser(learning_rate=learning_rate, optimiser=optimiser_name)

    # Compile model:
    model.compile(optimizer=optimiser,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=["accuracy"])
    
    # Fit model:
    # Configure callbacks:
    callbacks = [CustomLogCallback(model, x_val, y_val),
                 wandb.keras.WandbMetricsLogger()]
                 
    # Train model:
    model.fit(train_ds, 
              epochs=epochs,
              validation_data=valid_ds, 
              callbacks=callbacks, 
              verbose=0)

Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.


### Loop training function in sweep:

In [132]:
def sweep_train(config_defaults=None):
    
    # Initialise wandb and start run:
    with wandb.init(config=config_defaults):
        
        wandb.config.architecture = "DenseNet"
        wandb.config.dataset = "Chest X-Ray Images (Pneumonia)"
        wandb.config.num_classes = 3
        wandb.notes = "Sweeping through DenseNet model variations"

        model = build_network(wandb.config.fc_layer_size,
                              wandb.config.dropout_1,
                              wandb.config.dropout_2,
                              wandb.config.dropout_3)

        train(model=model, 
              batch_size=wandb.config.batch_size,
              epochs=wandb.config.epochs, 
              learning_rate=wandb.config.learning_rate,
              optimiser_name=wandb.config.optimiser)

### Initialise sweep and run agent:

In [133]:
sweep_id = wandb.sweep(sweep_config, project="PCXR-evaluating-densenet-model-variations")

Create sweep with ID: kjecbfle
Sweep URL: https://wandb.ai/marcus-lim/PCXR-evaluating-densenet-model-variations/sweeps/kjecbfle


In [134]:
wandb.agent(sweep_id, function=sweep_train, count=10)

wandb: Agent Starting Run: t4jk2lm2 with config:
wandb: 	batch_size: 64
wandb: 	dropout_1: 0.2
wandb: 	dropout_2: 0.8
wandb: 	dropout_3: 0.8
wandb: 	epochs: 10
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.0005869039959197109
wandb: 	optimiser: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 70ms/step
training loss : 1.8146089315414429 , training acc : 0.35058698058128357 , mcc score : 0.029230903834104538
aucroc score  : 0.5062130093574524 , f1 score     : 0.15713275969028473 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 67ms/step
training loss : 1.6669343709945679 , training acc : 0.3788687288761139 , mcc score : 0.20938295125961304
aucroc score  : 0.6241124272346497 , f1 score     : 0.34172549843788147 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 68ms/step
training loss : 1.6729501485824585 , training acc : 0.380736380815506 , mcc score : 0.4124642014503479
aucroc score  : 0.7800295948982239 , f1 score     : 0.5486

auc_roc,▁▃▆▇▇█████
epoch/accuracy,▁▂▂▄▅▆▆▇▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▄▃▃▂▂▁▁
epoch/val_accuracy,▁▃▅▇▇█████
epoch/val_loss,█▅▃▃▂▂▁▁▁▁
fmeasure,▁▃▆▇▇█████
mcc,▁▃▅▇▇█████
auc_roc,0.91746
epoch/accuracy,0.52588


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: brngfvbo with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.6
wandb: 	dropout_2: 0.2
wandb: 	dropout_3: 0.2
wandb: 	epochs: 10
wandb: 	fc_layer_size: 64
wandb: 	learning_rate: 0.003475100319230413
wandb: 	optimiser: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 68ms/step
training loss : 0.5920528769493103 , training acc : 0.7585378885269165 , mcc score : 0.6627742648124695
aucroc score  : 0.9221893548965454 , f1 score     : 0.7414975166320801 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 66ms/step
training loss : 0.5068715810775757 , training acc : 0.7929562330245972 , mcc score : 0.6490806341171265
aucroc score  : 0.9220414161682129 , f1 score     : 0.7474175095558167 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 66ms/step
training loss : 0.491942435503006 , training acc : 0.7982923984527588 , mcc score : 0.6932045221328735
aucroc score  : 0.9334319829940796 , f1 score     : 0.7858052253

...

Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 69ms/step
training loss : 0.9895410537719727 , training acc : 0.5635005235671997 , mcc score : 0.6366035342216492
aucroc score  : 0.9152366518974304 , f1 score     : 0.7504437565803528 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 64ms/step
training loss : 0.7798392176628113 , training acc : 0.6672891974449158 , mcc score : 0.6345846652984619
aucroc score  : 0.9084319472312927 , f1 score     : 0.7447826862335205 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 64ms/step
training loss : 0.7191582322120667 , training acc : 0.6851654052734375 , mcc score : 0.6459968686103821
aucroc score  : 0.9276626706123352 , f1 score     : 0.742470204

auc_roc,▃▁▇▅▇█▆███
epoch/accuracy,▁▅▆▇▇██▇██
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▂▂▁▂▁▁▁
epoch/val_accuracy,▁▂▃▅▅▆▄▆▆█
epoch/val_loss,█▆▅▃▂▂▃▂▂▁
fmeasure,▃▂▁▃▅▆▁▆▆█
mcc,▁▁▃▅▅▆▄▆▆█
auc_roc,0.93151
epoch/accuracy,0.72625


wandb: Agent Starting Run: pb39ak29 with config:
wandb: 	batch_size: 16
wandb: 	dropout_1: 0.2
wandb: 	dropout_2: 0.2
wandb: 	dropout_3: 0.6
wandb: 	epochs: 10
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.0004039459423685297
wandb: 	optimiser: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 70ms/step
training loss : 0.7202815413475037 , training acc : 0.7022411823272705 , mcc score : 0.6619464755058289
aucroc score  : 0.9303253889083862 , f1 score     : 0.7442891001701355 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 67ms/step
training loss : 0.5744310617446899 , training acc : 0.7676094174385071 , mcc score : 0.6741309762001038
aucroc score  : 0.9359467029571533 , f1 score     : 0.7437850832939148 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 67ms/step
training loss : 0.5164772272109985 , training acc : 0.7897545099258423 , mcc score : 0.68256676197052
aucroc score  : 0.9331360459327698 , f1 score     : 0.77299052476

auc_roc,▁▄▃▂▄▆▆▆█▆
epoch/accuracy,▁▅▆▇▇▇█▇▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▂▂▁▁▁▁▁
epoch/val_accuracy,▁▃▄▅▂█▇▅█▇
epoch/val_loss,█▄▃▂▅▂▂▂▁▂
fmeasure,▁▁▅▆▂▇▆▅█▇
mcc,▁▃▄▅▂█▇▅█▇
auc_roc,0.94024
epoch/accuracy,0.8167


In [135]:
# Close WandB run:
wandb.finish()